### OCI Data Science - pull data
This notebook provides tools and techniques to pull required data for the project

##### Use fastF1 API 
* Pull Lap, weather, car, results, and position data 




#### This notebook uses formula1conda which is a custom conda
fastf1 package requires python>=3.8 and OCI prebuild packages at this time mostly come with python==3.7.

#### Steps to build and publish a custom conda:
1. create a yaml file and list your packages 
2. in the terminal execute: `odsc conda create -f environment.yaml -n my-conda-env`
3. publish conda: initialize `odsc conda init -b  -n  -a {api_key | resource_principal}` and then publish `odsc conda publish -s my-conda-envv1_0`

In [1]:
path = '/home/datascience/WorkSpace/RedBull-Racining-TimeToPit/notebooks'
data_path = '../../RedBull/data/'

In [2]:
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import pickle
fastf1.Cache.enable_cache(data_path)

In [3]:
# schedule = fastf1.get_event_schedule(2022).to_dict()
# schedule = pd.DataFrame.from_dict(schedule)
# schedule.head(10)

In [4]:
# EventName = list(schedule['EventName'][(schedule.index<13) & (schedule.index>1)])

In [5]:
def get_lap_data(session, schedule, evnt, ses, EventDate):
    '''get lap data from a session and
       retun as a dataframe
    '''
    lap = session.laps.to_dict()
    lap = pd.DataFrame.from_dict(lap)
    lap['EventName'] =evnt
    lap['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    lap['session'] = ses
    lap['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return lap

In [6]:
def get_weather_data(session, schedule, evnt, ses,EventDate): 
    '''get weather data from a session and
       retun as a dataframe
    '''
    weather = session.weather_data.to_dict()
    weather = pd.DataFrame.from_dict(weather)
    weather['EventName'] =evnt
    weather['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    weather['session'] = ses
    weather['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return weather

In [7]:
def get_car_data(session, schedule, evnt, ses,EventDate):
    '''get car_data from a session and
       retun as a dataframe
    '''
    for ii in session.car_data:
        car_data = session.car_data[ii].to_dict()
        car_data = pd.DataFrame.from_dict(car_data)
        car_data['driver'] = ii
    car_data['EventName'] =evnt
    car_data['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    car_data['session'] = ses
    car_data['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return car_data

In [8]:
def get_position_data(session, schedule, evnt, ses, EventDate):
    '''get position_data from a session and
       retun as a dataframe
    '''    
    for ii in session.pos_data:
        position = session.pos_data[ii].to_dict()
        position = pd.DataFrame.from_dict(position)
        position['driver'] = ii
    position['EventName'] =evnt
    position['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    position['session'] = ses
    position['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return position

In [9]:
def get_results(session, schedule, evnt, ses, EventDate):
    '''get results data from a session and
       retun as a dataframe
    '''
    result = session.results.to_dict()
    result = pd.DataFrame.from_dict(result).reset_index()
    result['EventName'] =evnt
    result['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    result['session'] = ses
    result['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return result

In [11]:
%%time

sessionDateMap = {'Race': 'Session5Date',
          'Qualifying': 'Session4Date'}

for year in [2022, 2021, 2020, 2019]:
    
    laps =[]
    weathers =[]
    results = []
    car_data_ses =[]
    all_positions =[]
    
    sch = fastf1.get_event_schedule(year).to_dict()
    sch = pd.DataFrame.from_dict(sch)

#     schedule.drop(schedule[schedule['EventName'] == 'Pre-Season Test'].index, inplace = True)
    
    ## year 2022 doesn't have data after Jun
    if year == 2022:
        sch = sch[sch.index<12]
        
    EventName = [s for s in sch['EventName'] if "Grand" in s]
    
    for event in EventName:
        for session_type in ['Qualifying','Race']:
            
            eventDateColName = sessionDateMap[session_type]
            
            session = fastf1.get_session(year, event, session_type)
            session.load()
            
            ## get lap data for a session
            laps.append(get_lap_data(session, sch, event, 
                                     session_type, eventDateColName))
            
            ## get weather data for a session
            weathers.append(get_weather_data(session, sch, event, 
                                             session_type, eventDateColName))

            ## get car_data for a session
            car_data_ses.append(get_car_data(session, sch, event, 
                                             session_type, eventDateColName))
            
            ## get positions for a session
            all_positions.append(get_position_data(session, sch, event, 
                                                   session_type, eventDateColName))
            
            ## get results for a session
            results.append(get_results(session, sch, event, 
                                       session_type, eventDateColName))

    ## save all extracted session data in a year
    file = open(data_path+ 'laps_'+str(year)+'.pkl', 'wb')
    pickle.dump(laps,file)
    file = open(data_path+ 'weathers_'+str(year)+'.pkl', 'wb')
    pickle.dump(weathers,file)
    file = open(data_path+ 'results_'+str(year)+'.pkl', 'wb')
    pickle.dump(results,file)
    file = open(data_path+ 'car_data_ses_'+str(year)+'.pkl', 'wb')
    pickle.dump(car_data_ses,file)
    file = open(data_path+' all_positions_'+str(year)+'.pkl', 'wb')
    pickle.dump(all_positions,file)

## Concat all relevant pickle files and save them into a single pickle

In [12]:
def concat_data_and_save(data_source, data_path):
    '''
    e.g. data_source is weather
    1. finds all pkl files in the data_path folder
    2. concat all
    3. saves the in 'data_source'.pickle
    '''
    data =[]
    files = os.listdir(data_path)
    file = [x for x in files if x.endswith(".pkl") if data_source in x]

    for fl in file:
        file = open(data_path+fl,'rb')
        data.extend(pickle.load(file))
    data = pd.concat(data, axis=0)

    file=open(data_path+data_source+'_data.pickle','wb')
    pickle.dump(data,file)

In [13]:
concat_data_and_save('weather', data_path)
concat_data_and_save('car', data_path)
concat_data_and_save('positions', data_path)
concat_data_and_save('laps', data_path)
concat_data_and_save('results', data_path)